In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import sklearn
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
np.random.seed(42)

In [2]:
data = pd.read_csv("./input/train.csv")
LABELS = ['toxic', 'severe_toxic', 'obscene','threat', 'insult', 'identity_hate']

In [3]:
# Create train and Test Datasets
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

print(train.shape)
print(test.shape)

(127572, 8)
(31999, 8)


## Create Baseline ROC Score

In [4]:
def get_y_vec(df, column_name):
    return df[[column_name]].as_matrix().reshape((-1,))

In [17]:
#naive bayes classifier
#with C = 1.0 0.978368974747
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=4, ngram_range=(1,2), stop_words='english', lowercase=True, binary=False)),
    #('clf', SGDClassifier(loss='log', n_jobs=-1, max_iter = 1000, tol = 1e-3)),
    ('clf', LogisticRegression())
])

In [18]:
roc_scores = []
for label in LABELS:
    print("########### %s ####################\n"%(label))
    
    y_train = get_y_vec(df = train, column_name=label)
    X_train = train[['comment_text']].as_matrix().reshape((-1,))
    
    y_test = get_y_vec(df = test, column_name=label)
    X_test = test[['comment_text']].as_matrix().reshape((-1,))
    
    clf = text_clf.fit(X=X_train, y=y_train)
    y_ = clf.predict_proba(X_test)
    y_bin = y_[:,1] > 0.5
    print(classification_report(y_test, y_bin))
    print(confusion_matrix(y_test, y_bin))
    roc = roc_auc_score(y_test, y_[:, 1])
    print(roc)
    roc_scores.append(roc)
    print("########################################\n")
print(sum(roc_scores)/len(roc_scores))

########### toxic ####################

             precision    recall  f1-score   support

          0       0.96      1.00      0.98     28973
          1       0.93      0.59      0.72      3026

avg / total       0.96      0.96      0.95     31999

[[28843   130]
 [ 1249  1777]]
0.969732833968
########################################

########### severe_toxic ####################

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     31682
          1       0.58      0.22      0.32       317

avg / total       0.99      0.99      0.99     31999

[[31630    52]
 [  246    71]]
0.987162052232
########################################

########### obscene ####################

             precision    recall  f1-score   support

          0       0.98      1.00      0.99     30312
          1       0.92      0.61      0.73      1687

avg / total       0.98      0.98      0.97     31999

[[30227    85]
 [  660  1027]]
0.98356854569
#####

In [19]:
submission = pd.read_csv("./input/test.csv")

In [20]:
X_submission = submission[['comment_text']].as_matrix().reshape((-1,))   

In [21]:
#Create a model on full data set:
results = {}
for label in ['toxic', 'severe_toxic', 'obscene','threat', 'insult', 'identity_hate']:
    print("########### %s ####################\n"%(label))
    y_train = get_y_vec(df = data, column_name=label)
    X_train = data[['comment_text']].as_matrix().reshape((-1,))
    clf = text_clf.fit(X=X_train, y=y_train)
    y_ = clf.predict_proba(X_submission)
    results[label] = y_[:,1]

########### toxic ####################

########### severe_toxic ####################

########### obscene ####################

########### threat ####################

########### insult ####################

########### identity_hate ####################



In [22]:
LABELS = ['toxic', 'severe_toxic', 'obscene','threat', 'insult', 'identity_hate']
for  label in LABELS:
    submission[label] = results[label]

In [23]:
print(submission[0:3])
submission[0:3]['comment_text'].as_matrix()

                 id                                       comment_text  \
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...   
1  0000247867823ef7  == From RfC == \n\n The title is fine as it is...   
2  00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...   

      toxic  severe_toxic   obscene    threat    insult  identity_hate  
0  0.993404      0.133981  0.986226  0.020061  0.887003       0.199192  
1  0.007997      0.003114  0.005742  0.001839  0.008402       0.003472  
2  0.032635      0.003794  0.012982  0.001743  0.015946       0.004215  


array([ "Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,",
       '== From RfC == \n\n The title is fine as it is, IMO.',
       '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'], dtype=object)

In [24]:
headers = ["id"]+LABELS
print(headers)
submission[headers].head()

['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.993404,0.133981,0.986226,0.020061,0.887003,0.199192
1,0000247867823ef7,0.007997,0.003114,0.005742,0.001839,0.008402,0.003472
2,00013b17ad220c46,0.032635,0.003794,0.012982,0.001743,0.015946,0.004215
3,00017563c3f7919a,0.007182,0.002683,0.005110,0.001433,0.005362,0.001645
4,00017695ad8997eb,0.044297,0.003111,0.013151,0.002009,0.019218,0.002994


In [25]:
import time
submission[headers].to_csv("./input/submissions-%s.csv"%str(int(time.time())), index=False)